**3. Applied SQL: Exploring SpaceX Launch Records 📊**

This notebook marks the transition from data preparation to strategic business intelligence. After successfully acquiring and cleaning the data, this phase focuses on Exploratory Data Analysis (EDA) performed through structured SQL queries on a temporary SQLite database.

The Business Mandate: From Data to Decision
For the fictional competitor, Space Y, understanding the historical performance levers of the Falcon 9 is paramount for risk management and competitive pricing. Raw data alone is insufficient; we require calculated Key Performance Indicators (KPIs). Using SQL allows us to:

**Calculate Aggregates:** Determine precise success rates (the average of the binary Class column) across categorical variables like LaunchSite and Orbit.

**Uncover Trends:** Track the evolution of reusability success over time to identify technological maturation points.

**Validate Hypotheses:** Test initial assumptions, such as the correlation between PayloadMass and landing success probability.

This structured intelligence, derived efficiently through SQL, provides the vital quantitative foundation for analyzing risk and setting competitive launch prices before the final machine learning model is trained.

**💾 Phase 1:** Database Setup and Data Ingestion
The initial code blocks establish the in-memory SQLite database connection and ingest the prepared spacex_clean_data.csv file, converting our clean DataFrame into a searchable relational table.

**🔎 Phase 2**: Key Business Intelligence Queries
This phase executes the critical SQL statements needed to extract specific insights. For each query, the Python code will execute the SQL and display the resulting DataFrame for interpretation.

**Query A:** Overall Launch Success Rate
Goal: Determine the project's baseline by calculating the overall percentage of successful reusability attempts (Class = 1).

**Query B:** Success Rate Per Launch Site
Goal: Identify which primary launch site provides the best performance history for successful first-stage recovery. This highlights geographical and logistical factors influencing success.

**Query C:** Success Rate Per Orbit Type
Goal: Analyze how different orbital destinations (LEO, GTO, SSO, etc.) correlate with reusability success, providing insight into mission difficulty and fuel margins.

**Query D:** Launch Trends Over Time (Yearly Analysis)
Goal: Track the progress of the Falcon 9 program by calculating the average success rate and average PayloadMass for each year. This reveals the program's learning curve and maturation timeline.

**Query E:** Payload Mass vs. Outcome
Goal: Quantify the impact of mission payload size by finding the average PayloadMass for missions that succeeded (Class = 1) versus those that failed (Class = 0).

**✅ Phase 3:** Conclusion and Analytical Summary
After executing the queries above, this final section will interpret the resulting data and synthesize the key findings. The analytical summary confirms which features (e.g., Launch Site, Orbit, Time) are statistically significant, providing clear justification for their use in the subsequent machine learning models.

In [11]:
# Standard libraries for CSV handling, database, and table display
import csv
import sqlite3
import pandas as pd
import prettytable

# Configure PrettyTable default
prettytable.DEFAULT = 'DEFAULT'

# Load SQL extension for Colab
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


This cell establishes a connection to a local SQLite database named `my_data1.db`.

- `sqlite3.connect("/content/my_data1.db")`: This line connects to the database file. If the file doesn't exist, it will be created.
- `con = sqlite3.connect(...)`: The connection object is assigned to the variable `con`.
- `cur = con.cursor()`: A cursor object is created, which is used to execute SQL commands.
- `%sql sqlite:////content/my_data1.db`: This line uses the SQL magic command to connect the SQL extension to the same SQLite database, allowing you to run SQL queries directly in subsequent cells using `%%sql` or `%sql`.

In [12]:
# Connect to a local SQLite database file
con = sqlite3.connect("/content/my_data1.db")
cur = con.cursor()

# Optional: Use SQL magic in Colab for queries
%sql sqlite:////content/my_data1.db


This cell performs two main actions:

1.  **Loads data from a CSV file:**
    - `df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")`: This line reads the data from the provided URL into a pandas DataFrame named `df`. This DataFrame now holds the SpaceX launch records.

2.  **Stores the DataFrame into a SQLite table:**
    - `df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")`: This line takes the data from the `df` DataFrame and writes it into a SQLite table named `SPACEXTBL` within the connected database (`con`).
        - `if_exists='replace'`: If a table named `SPACEXTBL` already exists, it will be dropped and replaced with the new data.
        - `index=False`: This prevents pandas from writing the DataFrame index as a column in the SQL table.
        - `method="multi"`: This is a suggestion for how to insert the data, which can be more efficient for larger datasets.

The output `101` likely indicates the number of rows successfully inserted into the `SPACEXTBL` table.

In [13]:
# Load SpaceX launch CSV data into a DataFrame
df = pd.read_csv(
    "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
)

# Store data into SQLite table 'SPACEXTBL'
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")


101

This cell performs two main actions:

1.  **Drops and creates a clean table:**
    - `%sql DROP TABLE IF EXISTS SPACEXTABLE;`: This line uses the SQL magic command to drop the `SPACEXTABLE` if it already exists, ensuring a clean slate for the next step.
    - `%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null`: This line creates a new table named `SPACEXTABLE` by selecting all columns from the `SPACEXTBL` table, but only including rows where the `Date` column is not null. This cleans the data by removing entries without a date.

2.  **Attempts to check distinct launch sites:**
    - `%sql SELECT DISTINCT "Launch_Site" FROM space_mission;`: This line attempts to query the distinct values in the "Launch\_Site" column. However, it results in an error because the table name is mistyped as `space_mission` instead of the newly created `SPACEXTABLE`.

In [18]:
# Drop table if it exists
%sql DROP TABLE IF EXISTS SPACEXTABLE;

# Create a clean table with only non-null Date values
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

# Example: Check distinct launch sites (ensure correct table name)
%sql SELECT DISTINCT "Launch_Site" FROM space_mission;


 * sqlite:////content/my_data1.db
Done.
 * sqlite:////content/my_data1.db
Done.
 * sqlite:////content/my_data1.db
(sqlite3.OperationalError) no such table: space_mission
[SQL: SELECT DISTINCT "Launch_Site" FROM space_mission;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


This cell executes a SQL query to retrieve the first 5 rows from the `space_mission` table where the "Launch\_Site" starts with 'CCA%'.

- **`SELECT * FROM space_mission`**: Selects all columns and rows from the `space_mission` table.
- **`WHERE "Launch_Site" LIKE 'CCA%'`**: Filters the results to include only rows where the "Launch\_Site" value begins with "CCA".
- **`LIMIT 5`**: Limits the output to the first 5 matching rows.

The query result is then read into a pandas DataFrame named `df_cca` and printed. This helps to quickly inspect a subset of the data from the CCA launch sites.

In [24]:
query = """
SELECT *
FROM space_mission
WHERE "Launch_Site" LIKE 'CCA%'
LIMIT 5;
"""
df_cca = pd.read_sql_query(query, conn)
print(df_cca)

         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

**Results Interpretation:**

The output displays the first 5 rows of data for launches from sites starting with 'CCA'. This provides a quick look at the structure and content of the data for these specific launch sites, including details like the date, time, booster version, payload, payload mass, orbit, customer, mission outcome, and landing outcome for these initial missions.

In [25]:
query = """
SELECT SUM("Payload_Mass") AS Total_Payload_Mass
FROM space_mission
WHERE "Customer" = 'NASA (CRS)';
"""
result = pd.read_sql_query(query, conn)
print(result)

   Total_Payload_Mass
0                 0.0


This cell executes a SQL query to calculate the total payload mass for all launches where the customer is 'NASA (CRS)'.

- **`SELECT SUM("Payload_Mass") AS Total_Payload_Mass`**: Calculates the sum of the "Payload\_Mass" column and names the result "Total\_Payload\_Mass".
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Customer" = 'NASA (CRS)'`**: Filters the results to include only rows where the "Customer" column is exactly 'NASA (CRS)'.

The query result, the total payload mass for NASA (CRS) missions, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays the total payload mass for all SpaceX missions with 'NASA (CRS)' as the customer. The value shown in the `Total_Payload_Mass` column represents the cumulative weight of all payloads sent to space for NASA's Commercial Resupply Services program.

In [26]:
query = """
SELECT AVG("Payload_Mass") AS Average_Payload_Mass
FROM space_mission
WHERE "Booster_Version" = 'F9 v1.1';
"""
result = pd.read_sql_query(query, conn)
print(result)

   Average_Payload_Mass
0                   0.0


This cell executes a SQL query to calculate the average payload mass for all launches where the "Booster\_Version" is 'F9 v1.1'.

- **`SELECT AVG("Payload_Mass") AS Average_Payload_Mass`**: Calculates the average of the "Payload\_Mass" column and names the result "Average\_Payload\_Mass".
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Booster_Version" = 'F9 v1.1'`**: Filters the results to include only rows where the "Booster\_Version" is 'F9 v1.1'.

The query result, the average payload mass for 'F9 v1.1' booster launches, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays the average payload mass for all SpaceX missions that used the 'F9 v1.1' booster version. The value in the `Average_Payload_Mass` column represents the typical weight of payloads launched with this specific booster.

In [27]:
query = """
SELECT MIN("Date") AS First_Successful_RTLS
FROM space_mission
WHERE "Landing_Outcome" = 1 AND "Booster_Landing" = 'RTLS';
"""
result = pd.read_sql_query(query, conn)
print(result)

  First_Successful_RTLS
0                  None


This cell executes a SQL query to find the date of the first successful RTLS (Return to Launch Site) landing.

- **`SELECT MIN("Date") AS First_Successful_RTLS`**: Selects the minimum value from the "Date" column and names the result "First\_Successful\_RTLS".
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Landing_Outcome" = 1 AND "Booster_Landing" = 'RTLS'`**: Filters the results to include only rows where the "Landing\_Outcome" is 1 (indicating success) and the "Booster\_Landing" method is 'RTLS'.

The query result, the date of the first successful RTLS landing, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays the date of the very first successful RTLS landing recorded in the dataset. If the output shows `None` or an empty result, it means there were no successful RTLS landings found within the filtered data.

In [29]:
query = """
SELECT "Booster_Version"
FROM space_mission
WHERE "Landing_Outcome" = 1
  AND "Booster_Landing" = 'ASDS'
  AND "Payload_Mass" > 4000
  AND "Payload_Mass" < 6000;
"""
result = pd.read_sql_query(query, conn)
print(result)

Empty DataFrame
Columns: [Booster_Version]
Index: []


This cell executes a SQL query to find the "Booster\_Version" for launches that meet specific criteria:

- **`SELECT "Booster_Version"`**: Selects the "Booster\_Version" column.
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Landing_Outcome" = 1`**: Filters for launches where the "Landing\_Outcome" is 1 (indicating success).
- **`AND "Booster_Landing" = 'ASDS'`**: Further filters for launches where the "Booster\_Landing" method was 'ASDS' (Autonomous Spaceport Drone Ship).
- **`AND "Payload_Mass" > 4000`**: Filters for launches where the "Payload\_Mass" is greater than 4000 kg.
- **`AND "Payload_Mass" < 6000`**: Filters for launches where the "Payload\_Mass" is less than 6000 kg.

The query result, a list of booster versions that satisfy these conditions, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays the "Booster\_Version" for any launches that had a successful ASDS landing with a payload mass between 4000 kg and 6000 kg. If the output shows an empty DataFrame, it means no launches in the dataset met all of these criteria.

In [30]:
query = """
SELECT "Landing_Outcome", COUNT(*) AS Total
FROM space_mission
GROUP BY "Landing_Outcome";
"""
result = pd.read_sql_query(query, conn)
print(result)

           Landing_Outcome  Total
0       Controlled (ocean)      5
1                  Failure      3
2     Failure (drone ship)      5
3      Failure (parachute)      2
4               No attempt     21
5              No attempt       1
6   Precluded (drone ship)      1
7                  Success     38
8     Success (drone ship)     14
9     Success (ground pad)      9
10    Uncontrolled (ocean)      2


This cell executes a SQL query to count the number of occurrences for each unique "Landing\_Outcome".

- **`SELECT "Landing_Outcome", COUNT(*) AS Total`**: Selects the "Landing\_Outcome" column and counts the number of rows for each outcome, naming the count "Total".
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`GROUP BY "Landing_Outcome"`**: Groups the results by the values in the "Landing\_Outcome" column, so the count is done for each distinct outcome.

The query result, a table showing each landing outcome and its total count, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays a summary of all the different "Landing\_Outcome" values present in the dataset and the total number of times each outcome occurred. This provides a clear overview of the distribution of landing results for SpaceX missions.

In [32]:
query = """
SELECT "Booster_Version"
FROM space_mission
WHERE "Payload_Mass" = (
    SELECT MAX("Payload_Mass")
    FROM space_mission
);
"""
result = pd.read_sql_query(query, conn)
print(result)

    Booster_Version
0    F9 v1.0  B0003
1    F9 v1.0  B0004
2    F9 v1.0  B0005
3    F9 v1.0  B0006
4    F9 v1.0  B0007
..              ...
96     F9 B5B1062.1
97    F9 B5B1061.1 
98     F9 B5B1063.1
99   F9 B5 B1049.7 
100  F9 B5 B1058.4 

[101 rows x 1 columns]


This cell executes a SQL query to find the "Booster\_Version" for launches that carried the maximum payload mass in the dataset.

- **`SELECT "Booster_Version"`**: Selects the "Booster\_Version" column.
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Payload_Mass" = (SELECT MAX("Payload_Mass") FROM space_mission)`**: This is a subquery that finds the maximum value in the "Payload\_Mass" column. The outer query then filters the `space_mission` table to include only rows where the "Payload\_Mass" is equal to this maximum value.

The query result, a list of booster versions that carried the maximum payload, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays the booster versions used for all launches that carried the heaviest payload recorded in the dataset. If multiple launches carried the same maximum payload mass, all corresponding booster versions will be listed.

In [33]:
query = """
SELECT
    substr("Date", 6, 2) AS Month,
    "Booster_Version",
    "Launch_Site",
    "Landing_Outcome",
    "Booster_Landing"
FROM space_mission
WHERE "Landing_Outcome" = 0
  AND "Booster_Landing" = 'ASDS'
  AND substr("Date", 1, 4) = '2015';
"""
result = pd.read_sql_query(query, conn)
print(result)


Empty DataFrame
Columns: [Month, Booster_Version, Launch_Site, Landing_Outcome, "Booster_Landing"]
Index: []


This cell executes a SQL query to retrieve information about launches in the year 2015 that had a failed ASDS (Autonomous Spaceport Drone Ship) landing.

- **`SELECT substr("Date", 6, 2) AS Month, "Booster_Version", "Launch_Site", "Landing_Outcome", "Booster_Landing"`**: Selects the month from the "Date" column (using `substr`), along with the "Booster\_Version", "Launch\_Site", "Landing\_Outcome", and "Booster\_Landing" columns.
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Landing_Outcome" = 0`**: Filters for launches where the "Landing\_Outcome" is 0 (indicating failure).
- **`AND "Booster_Landing" = 'ASDS'`**: Further filters for launches where the "Booster\_Landing" method was 'ASDS'.
- **`AND substr("Date", 1, 4) = '2015'`**: Filters for launches that occurred in the year 2015 (by extracting the year from the "Date" column using `substr`).

The query result, a list of launches in 2015 with failed ASDS landings, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays the details for any launches in 2015 that experienced a failed ASDS landing. If the output shows an empty DataFrame, it means there were no launches in 2015 that met these specific criteria.

In [35]:
query = """
SELECT
    "Landing_Outcome",
    "Booster_Landing",
    COUNT(*) AS Outcome_Count
FROM space_mission
WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY "Landing_Outcome", "Booster_Landing"
ORDER BY Outcome_Count DESC;
"""
result = pd.read_sql_query(query, conn)
print(result)

          Landing_Outcome "Booster_Landing"  Outcome_Count
0              No attempt   Booster_Landing             10
1    Failure (drone ship)   Booster_Landing              5
2    Success (drone ship)   Booster_Landing              5
3      Controlled (ocean)   Booster_Landing              3
4    Success (ground pad)   Booster_Landing              3
5     Failure (parachute)   Booster_Landing              2
6    Uncontrolled (ocean)   Booster_Landing              2
7  Precluded (drone ship)   Booster_Landing              1


This cell executes a SQL query to count the number of occurrences for each combination of "Landing\_Outcome" and "Booster\_Landing" within a specified date range.

- **`SELECT "Landing_Outcome", "Booster_Landing", COUNT(*) AS Outcome_Count`**: Selects the "Landing\_Outcome" and "Booster\_Landing" columns and counts the number of rows for each unique combination, naming the count "Outcome\_Count".
- **`FROM space_mission`**: Specifies that the data is being retrieved from the `space_mission` table.
- **`WHERE "Date" BETWEEN '2010-06-04' AND '2017-03-20'`**: Filters the results to include only launches that occurred between the specified start and end dates.
- **`GROUP BY "Landing_Outcome", "Booster_Landing"`**: Groups the results by both "Landing\_Outcome" and "Booster\_Landing", so the count is done for each distinct combination of these two columns.
- **`ORDER BY Outcome_Count DESC`**: Orders the results in descending order based on the "Outcome\_Count", showing the most frequent outcomes first.

The query result, a table showing each combination of landing outcome and booster landing method within the date range and its count, is then read into a pandas DataFrame named `result` and printed.

**Results Interpretation:**

The output displays a summary of the different combinations of "Landing\_Outcome" and "Booster\_Landing" within the specified date range, along with the total number of times each combination occurred. The results are ordered by the count, allowing for easy identification of the most common landing scenarios during this period.